In [ ]:
import requests
import pandas as pd
import os

### grab info from api

In [ ]:
#https://members-api.parliament.uk/index.html
#set variables
skip = 0
take = 20
all_members = []
current_member = 'true'
house = 1 #1 for commons, 2 for lords

#iterate through each page
while True:
    url = (f"https://members-api.parliament.uk/api/Members/Search?" +
    f"IsCurrentMember={current_member}&" +
    f"house={house}&" +
    f"skip={skip}&take={take}")
    response = requests.get(url)

    if response.status_code == 200: #if has found records
        data = response.json()
        members = data.get('items', [])
        if not members:
            break
        all_members.extend(members)
        print(f'acquired members #{skip}-{skip+take}')
        skip += take
    else:
        print(f"request failed: {response.status_code}")
        break

### format + hone data

In [ ]:
#put desired data in dataframe
data = []
for member in all_members:
    _id = member['value']['id']
    name = member['value']['nameDisplayAs']
    party = member['value']['latestParty']['name']
    party_colour = f"#{member['value']['latestParty']['backgroundColour']}"
    constituency = member['value']['latestHouseMembership']['membershipFrom']
    start_date = member['value']['latestHouseMembership']['membershipStartDate']
    contact = f"https://members.parliament.uk/member/{member['value']['id']}/contact"
    image_url = member['value']['thumbnailUrl']
    
    data.append({
        'id': _id,
        'name': name,
        'party': party,
        'party_colour': party_colour,
        'constituency': constituency,
        'start_date': start_date,
        'contact': contact,
        'image_url': image_url
    })
df = pd.DataFrame(data)

In [ ]:
#make dates more readable
df['start_date_string'] = pd.to_datetime(df['start_date']).dt.strftime('%d %B, %Y')
df['start_date_string'] = df['start_date_string'].str.lstrip('0')

In [ ]:
#make column for initial order (last name alphabetical)
df = df.reset_index()
df = df.rename(columns={'index':'order'})

In [ ]:
#reorder
df = df[['order','id','name','party','party_colour','constituency','start_date','start_date_string','contact','image_url']]

In [ ]:
df.head()

### download images locally

In [ ]:
#make thumbnails folder
if not os.path.exists(images_folder):
    os.makedirs(images_folder)
images_folder = 'table/images/thumbnails'
if not os.path.exists(images_folder):
    os.makedirs(images_folder)

In [ ]:
#iterate through + download images
for idx, row in df.iterrows():
    filename = f"{row['id']}_thumbnail.jpg"
    image_path = os.path.join(images_folder, filename)
    with requests.get(row['image_url'], stream=True) as request:
        with open(image_path, 'wb') as file:
            file.write(request.content)

In [ ]:
#resize images
from PIL import Image
from io import BytesIO

for idx, row in df.iterrows():
    filename = f"{row['id']}_thumbnail.jpg"
    image_path = os.path.join(images_folder, filename)
    
    with requests.get(row['image_url'], stream=True) as request:
        if request.status_code == 200:
            image = Image.open(BytesIO(request.content))
            if image.mode in ('RGBA', 'P'):
                image = image.convert('RGB')
            aspect_ratio = image.width / image.height
            new_height = int(180 / aspect_ratio)
            image = image.resize((180, new_height), Image.LANCZOS)
            image.save(image_path, format='JPEG')

### export dataframe

In [ ]:
df.to_json('table/data/data.json', orient='table', index=False)

In [ ]:
#export csv download
dfExport = df[['name','party','constituency','start_date_string','contact','image_url']]
dfExport.to_csv('table/data/members_of_parliament_2024.csv')